In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
file_path = "patient_heart_rate.csv"


file_path = 'patient_heart_rate.csv'
names = ["Id", "Name", "Age", "Weight", "m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]

# Thêm on_bad_lines='skip' để bỏ qua dòng lỗi
df = pd.read_csv(file_path, names=names, encoding='utf-8-sig', on_bad_lines='skip')

print("Dữ liệu gốc:")
print(df.head(15))





Dữ liệu gốc:
      Id            Name   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218
0    1.0    Mickéy Mousé  56.0       70kgs    72    69    71     -     -     -
1    2.0     Donald Duck  34.0   154.89lbs     -     -     -    85    84    76
2    3.0      Mini Mouse  16.0         NaN     -     -     -    65    69    72
3    4.0  Scrooge McDuck   NaN       78kgs    78    79    72     -     -     -
4    5.0    Pink Panther  54.0  198.658lbs     -     -     -    69   NaN    75
5    6.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
6    7.0    Dewey McDuck  19.0       56kgs     -     -     -    71    78    75
7    8.0      Scööpy Doo  32.0       78kgs    78    76    75     -     -     -
8    NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
9    NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
10   9.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
11  10.0    Louie McDuck  12.0       45

# LAB 3

Vấn đề 1: Xử lý thiếu Header

In [74]:
# Đọc dữ liệu, không có header
names = ["Id", "Name", "Age", "Weight", "m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]
df = pd.read_csv(file_path, names=names)

print("Dữ liệu gốc:")
print(df.head(15))


Dữ liệu gốc:
      Id            Name   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218
0    1.0    Mickéy Mousé  56.0       70kgs    72    69    71     -     -     -
1    2.0     Donald Duck  34.0   154.89lbs     -     -     -    85    84    76
2    3.0      Mini Mouse  16.0         NaN     -     -     -    65    69    72
3    4.0  Scrooge McDuck   NaN       78kgs    78    79    72     -     -     -
4    5.0    Pink Panther  54.0  198.658lbs     -     -     -    69   NaN    75
5    6.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
6    7.0    Dewey McDuck  19.0       56kgs     -     -     -    71    78    75
7    8.0      Scööpy Doo  32.0       78kgs    78    76    75     -     -     -
8    NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
9    NaN             NaN   NaN         NaN   NaN   NaN   NaN   NaN   NaN   NaN
10   9.0     Huey McDuck  52.0      189lbs     -     -     -    68    75    72
11  10.0    Louie McDuck  12.0       45

Vấn đề 2: Tách cột "Name"


In [75]:
# Tách cột Name thành Firstname và Lastname
df[['Firstname', 'Lastname']] = df['Name'].str.split(expand=True)

# Xóa cột Name ban đầu
df = df.drop('Name', axis=1)

print("\nSau khi tách tên:")
print(df[['Firstname', 'Lastname']].head())



Sau khi tách tên:
  Firstname Lastname
0    Mickéy    Mousé
1    Donald     Duck
2      Mini    Mouse
3   Scrooge   McDuck
4      Pink  Panther


Vấn đề 3: Thống nhất đơn vị đo lường cột "Weight"


In [76]:
def clean_weight(x):
    if pd.isna(x):
        return x
    x = str(x)
    if 'kgs' in x:
        return float(x.replace('kgs', ''))
    elif 'lbs' in x:
        return float(x.replace('lbs', '')) * 0.453592
    else:
        # Trường hợp không có đơn vị hoặc dữ liệu lỗi, cố gắng chuyển sang float
        try:
            return float(x)
        except ValueError:
            return np.nan

df['Weight'] = df['Weight'].apply(clean_weight)

print("\nSau khi chuẩn hóa cân nặng (kg):")
print(df['Weight'].head())



Sau khi chuẩn hóa cân nặng (kg):
0    70.000000
1    70.256865
2          NaN
3    78.000000
4    90.109680
Name: Weight, dtype: float64


Vấn đề 4: Xóa dòng dữ liệu rỗng

In [77]:
# Xóa các dòng mà tất cả các cột đều là NaN
df.dropna(how='all', inplace=True)


Vấn đề 5: Xử lý dữ liệu trùng lặp


In [78]:
# Xóa dòng trùng lặp dựa trên các cột định danh chính
df.drop_duplicates(subset=['Firstname', 'Lastname', 'Age', 'Weight'], inplace=True)

print("\nSố lượng dòng sau khi xóa trùng lặp:", len(df))



Số lượng dòng sau khi xóa trùng lặp: 14


Vấn đề 6: Xử lý lỗi Non-ASCII

In [79]:
df['Firstname'] = df['Firstname'].replace({r'[^\x00-\x7F]+':''}, regex=True)
df['Lastname'] = df['Lastname'].replace({r'[^\x00-\x7F]+':''}, regex=True)

print("Sau khi xử lý ký tự lạ:")
print(df[['Firstname', 'Lastname']].head())


Sau khi xử lý ký tự lạ:
  Firstname Lastname
0     Micky     Mous
1    Donald     Duck
2      Mini    Mouse
3   Scrooge   McDuck
4      Pink  Panther


Vấn đề 7: Xử lý Missing Values (Age & Weight)


In [80]:
# 1. Xóa dòng nếu thiếu cả Age và Weight
df.dropna(subset=['Age', 'Weight'], how='all', inplace=True)

# 2. Điền dữ liệu thiếu cho Age bằng Mean
# Chuyển cột Age sang kiểu số (nếu chưa phải)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Age'] = df['Age'].fillna(df['Age'].mean())

# (Tùy chọn) Điền dữ liệu thiếu cho Weight bằng Mean nếu cần thiết, 
# tuy nhiên đề bài nhấn mạnh vào Age.
df['Weight'] = df['Weight'].fillna(df['Weight'].mean())

print("\nSau khi xử lý dữ liệu thiếu:")
print(df[['Age', 'Weight']].head())



Sau khi xử lý dữ liệu thiếu:
    Age     Weight
0  56.0  70.000000
1  34.0  70.256865
2  16.0  71.474199
3  36.1  78.000000
4  54.0  90.109680


Vấn đề 8: Chuẩn hóa cấu trúc bảng (Melt Data)


In [81]:
# Melt dữ liệu
df = pd.melt(df, id_vars=['Id', 'Age', 'Weight', 'Firstname', 'Lastname'], 
             value_name='PulseRate', var_name='sex_and_time')

# Tách cột sex_and_time thành Sex và Time
# Regex: (\D) lấy ký tự đầu (m/f), (\d{2}) lấy 2 số tiếp, (\d{2}) lấy 2 số cuối
tmp_df = df['sex_and_time'].str.extract(r'(\D)(\d{2})(\d{2})')
tmp_df.columns = ['Sex', 'hours_lower', 'hours_upper']

# Tạo cột Time
tmp_df['Time'] = tmp_df['hours_lower'] + '-' + tmp_df['hours_upper']

# Gép lại vào dataframe chính
df = pd.concat([df, tmp_df], axis=1)

# Xóa các cột thừa
df = df.drop(['sex_and_time', 'hours_lower', 'hours_upper'], axis=1)

# Xử lý cột PulseRate: thay thế '-' bằng NaN và xóa dòng không có dữ liệu nhịp tim
df['PulseRate'] = df['PulseRate'].replace('-', np.nan)
df.dropna(subset=['PulseRate'], inplace=True)

print("\nKết quả cuối cùng (5 dòng đầu):")
print(df.head())

# Lưu file kết quả
df.to_csv('patient_heart_rate_clean.csv', index=False)



Kết quả cuối cùng (5 dòng đầu):
      Id   Age     Weight Firstname Lastname PulseRate Sex   Time
0    1.0  56.0  70.000000     Micky     Mous        72   m  00-06
3    4.0  36.1  78.000000   Scrooge   McDuck        78   m  00-06
7    8.0  32.0  78.000000      Scpy      Doo        78   m  00-06
9   11.0  36.1  60.000000     Henry      Nam        78   m  00-06
10  12.0  34.0  71.474199    Michel     Long        65   m  00-06
